In [1]:
import pandas as pd
import tarfile
import json
import os
from pandas import json_normalize

from lib import *
from train import *
from demo import *
import re
from datetime import datetime

In [2]:
def parse_date(filename):
    basename = os.path.basename(filename)
    match = re.search(r'reports_(\w+)(\d+)_(\d+)', basename)
    if not match:
        raise ValueError(f"Konnte kein Datum aus dem Dateinamen extrahieren: {basename}")

    month, day, year = match.groups()

    # Konvertiere den Monatsnamen in eine Zahl
    month_num = datetime.strptime(month, '%b').month

    # Erstelle ein vollständiges Datum
    date_str = f"{year}-{month_num:02d}-{int(day):02d}"
    return datetime.strptime(date_str, "%Y-%m-%d")

In [3]:
def read_json_from_tar_and_convert_to_df(tar_file):
    json_data = []
    with tarfile.open(tar_file, 'r:gz') as tar:
        for member in tar.getmembers():
            if member.isfile() and member.name.endswith('.json'):
                file = tar.extractfile(member)
                data = json.load(file)
                json_data.append(data)
    df = pd.DataFrame(json_normalize(json_data))
    return df

In [13]:
data_list = []
directory = 'data/protondb-data/reports'
df = None
for file in os.listdir(directory):
    if file.endswith('.tar.gz'):
        print(f"Extracting file {file}")
        date = parse_date(file)
        if date >= datetime.strptime("2022-03-01", "%Y-%m-%d"):
            file_path = os.path.join(directory, file)
            data =  read_json_from_tar_and_convert_to_df(file_path)
            if df is None:
                df = data
            else:
                df = pd.concat([df, data])

In [ ]:
df = pd.DataFrame(json_normalize(data_list))